# Nodding Data Reduction

This tutorial shows the reduction path of a nodding observation with a multi-beam receiver, the K-band Focal Plane Array (KFPA).

Similar to a position-switched observation, two beams are selected for simultaneous observing (though the receiver can have more than two beams). In the first scan BEAM1 is looking at the source, while BEAM2 is looking at an assumed OFF position. In the next scan, BEAM2 will be looking at the source, while BEAM1 is looking at (another) OFF position. This will result in two position-switched solutions, which are then averaged for the final spectrum. 

One advantage of this observing mode is that the telescope is always ON source, bringing the noise down by $\sqrt{2}$ compared to a classic position switched observation. Minus a small amount of slewing time of course. However, the beam separation in the receiver should be large enough to ensure that the off position is not on the source. Otherwise, a proper Position Switching observation is needed with a large enough offset.

In the observations used in this tutorial there are also position switched observations of the same source, so we can compare the results of using beam nodding versus position switching.

The data in this tutorial were also presented in https://gbtdocs.readthedocs.io/en/latest/how-tos/data_reduction/gbtidl.html#basic-nodding in a similar GBTIDL data reduction.

You can find a copy of this tutorial as a Jupyter notebook [here](https://github.com/GreenBankObservatory/dysh/blob/main/notebooks/examples/nodding.ipynb) or download it by right clicking  <a href="https://raw.githubusercontent.com/GreenBankObservatory/dysh/refs/heads/main/notebooks/examples/nodding.ipynb" download>here</a> and selecting "Save Link As".

## Background

The spectral line observed here is the NH$_3$ (1,1) line at 23.69 GHz with the KFPA receiver. This receiver has 7 beams: one central beam and six beams in a roughly hexagonal pattern around the central beam. The source is a position in the W3 cloud, a roughly two degree sized Giant Molecular Cloud (GMC) with active star formation. See also https://herscheltelescope.org.uk/results/w3-star-forming-region/

## Loading Modules
We start by loading the modules we will use for the data reduction. 

For display purposes, we use the static (non-interactive) `matplotlib` backend in this tutorial. However, you can tell `matplotlib` to use the `ipympl` backend to enable interactive plots. This is only needed if working on `jupyter` lab or notebook.

In [ ]:
# Set interactive plots in jupyter.
#%matplotlib ipympl

# These modules are required for the data reduction.
from dysh.log import init_logging
from dysh.fits.gbtfitsload import GBTFITSLoad
from astropy import units as u

# These modules are only used to download the data.
from pathlib import Path
from dysh.util.download import from_url

## Setup Logging

`dysh` uses a logger to communicate. If you are working in the command line, then the logging is setup for you. If you are working in a jupyter lab instance, then you need to set it up. You can do so using the `init_logging` function imported above. As an argument, `init_logging` takes a number, the verbosity `level`. `level` 0 is for error messages only, 1 for warning, 2 for info and 3 for debug. Here we set it to `level` 2. 

In [ ]:
init_logging(2)

## Data Retrieval

Download the example SDFITS data, if necessary.

In [ ]:
url = "http://www.gb.nrao.edu/dysh/example_data/nod-KFPA/data/TGBT22A_503_02.raw.vegas.trim.fits"
savepath = Path.cwd() / "data"
savepath.mkdir(exist_ok=True) # Create the data directory if it does not exist.
filename = from_url(url, savepath)

## Data Loading

Next, we use `GBTFITSLoad` to load the data, and then its `summary` method to inspect its contents.

This trimmed dataset is an extraction from a much larger dataset (19GB) and can take some time to load if it's the first time.

In [ ]:
sdfits = GBTFITSLoad(filename)

In [ ]:
sdfits.summary()

## Data Reduction

### Nodding Data Reduction

We start calibrating the Nod observations in scans 62 and 63. We use the `GBTFITSLoad.getnod` method to calibrate the data. This method will automatically find a pair of Nod scans given one scan number. It will also automatically figure out which feeds where used during the nodding observations. The return of `getnod` is a `ScanBlock` with at least two `NodScan` in it.

In [ ]:
nodsb = sdfits.getnod(scan=62, ifnum=0, plnum=0)
nodsb

Each `NodScan` holds all of the calibrated integrations for each feed used during the Nod observations.
We can query the `NodScan` objects for information about the data, such as what is the system temperature, in K, or the exposure time, in seconds. `NodScan` is a sub class of a `Scan` object.

In [ ]:
nodsb[0].tsys, nodsb[1].tsys

In [ ]:
nodsb[0].exposure, nodsb[1].exposure

In [ ]:
nodsb[0].fdnum, nodsb[1].fdnum

From the above we see that the beams used for nodding had fdnum of 2 and 6.

#### Inspecting Integrations

To access the calibrated integrations we use the `calibrated` method of a `Scan` object. The return is a `Spectrum` object with the calibrated data. The argument to `calibrated` is the integration number.

In [ ]:
nod_int = nodsb[0].getspec(0)
nod_int

`Spectrum` objects have convenience functions to plot, smooth, and remove baselines, among others. Here we use the `plot` function to display the calibrated data for the first integration.

In [ ]:
nod_int.plot()

The plot shows a noise-like signal since it is only one 1 s integration. To see more details, and potentially a signal, we must time average the data.

#### Time Averaging

To time average we can use the `timeaverage` function. We can use this function directly from a `ScanBlock`, in which case all of the data in the `ScanBlock` will be time averaged, or from a `Scan`, and only average the data inside the `Scan`. Here we will average all the data in the `ScanBlock`.

By default time averaging uses the following weights: 
$$
\frac{T^{2}_{sys}}{\Delta\nu\Delta t}
$$
with $T_{sys}$ the system temperature, $\Delta\nu$ the channel width and $\Delta t$ the integration time. In `dysh` these are set using `weights='tsys'` (the default).

The return of `timeaverage` is a `Spectrum` object.

In [ ]:
nod_ta = nodsb.timeaverage()
nod_ta

Now we plot the time average.

In [ ]:
nod_ta.plot()

We see hints of a line. We can further reduce our data to bring out the signal from the noise.

#### Smoothing

One way of reducing the noise in the time average is by smoothing the data along the spectral axis. This is done using the `Spectrum.smooth` function. The first argument is the kernel, and the second the width of the kernel in channels. The available kernels are a boxcar, Gaussian and a Hanning window. We use a boxcar with a width of 51 channels.

In [ ]:
nod_ta_smooth = nod_ta.smooth('box', 51)

In [ ]:
nod_ta_smooth.plot(xaxis_unit="GHz")

The signal is clear now. 

#### Statistics

We can quantify the reduction in the noise using the `Spectrum.stats` function. To reduce the bias in the statistics, we will slice the spectrum to avoid the bandpass roll off channels. We use a channel range between 23.687 and 23.694 GHz.

In [ ]:
s = slice(23.687*u.GHz, 23.694*u.GHz)

In [ ]:
nod_ta_smooth[s].stats()

In [ ]:
nod_ta[s].stats()

Before smoothing the rms was 339 mK, after smoothing it went down to 46 mK, so the reduction in the noise is 6% higher than $\sqrt{51}$.

#### Baseline Subtraction

Now we will subtract a baseline from the data. We use the `Spectrum.baseline` function to do it. We will ignore the edge channels and the region with line emission during the baseline fitting. This is specified with the `exclude` or `include` argument of `baseline`. Here we use `include`. Since the line free channels seem to have a flat frequency response, away from the window edges, we use an order 1 polynomial as our baseline model.

In [ ]:
include = [(23.687*u.GHz, 23.694*u.GHz),
           (23.700*u.GHz, 23.705*u.GHz)
          ]
model = "poly"
order = 1

nod_ta_smooth.baseline(order, model=model, include=include, remove=True)

In [ ]:
nod_ta_smooth.plot()

Now we will proceed with the calibration of the OnOff observations for comparison.

### OnOff Data Reduction

Scans 60 and 61 contain OnOff observations (position switched). For the 7-beam KFPA receiver, the central beam (`fdnum=0`) will be the source tracking beam for the OnOff observations. As with the Nod calibration, dysh knows how to pair OnOff observations given a scan number. We use the `GBTFITSLoad.getps` function to calibrate position switched observations. As with `getnod`, the return of `getps` is a `ScanBlock`, but this time containing `PSScan` objects, which are also sub classes of the `Scan` class, so they share many of their functions and properties.

In [ ]:
pssb = sdfits.getps(scan=60, plnum=0, ifnum=0, fdnum=0)
pssb

We time average, smooth and baseline subtract the calibrated data. We use a chain of functions for the time averaging and smoothing.

In [ ]:
ps_ta_smooth = pssb.timeaverage().smooth("box", 51)
ps_ta_smooth.baseline(order, model=model, include=include, remove=True)

In [ ]:
ps_ta_smooth.plot(ymin=-0.2)

## Comparing the Methods

Both the Nod and OnOff scans shows a signal, How do the properties of the spectra compare?

In [ ]:
# Line free rms
nod_ta_smooth[s].stats()["rms"], ps_ta_smooth[s].stats()["rms"]

The noise in the Nod spectrum is lower than that in the OnOff one. This makes sense since the exposure time in the Nod observations is almost twice as long as in the OnOff ones, for the same observing time. If that is the only factor, we would expect the noise in the Nod spectrum to be a factor $\sqrt{2}$ lower than the OnOff one. Let's check.

In [ ]:
ps_ta_smooth[s].stats()["rms"]/nod_ta_smooth[s].stats()["rms"]/2**0.5

The ratio is 0.85, meaning that the noise reduction was 15% less than expected. This is likely because the system temperature of the feeds is different. Let's check.

In [ ]:
ps_ta_smooth.meta["TSYS"]/nod_ta_smooth.meta["TSYS"]

The system temperature of the feed used for the OnOff observations is 15% lower than that of the feeds used for the Nod. This explains the observed difference in the noise.

The line peak is consistent between the Nod and OnOff spectra.

In [ ]:
# Line peak
nod_ta_smooth.stats()["max"], ps_ta_smooth.stats()["max"]